In [1]:
import pandas as pd
import sqlite3

In [6]:
egg_df = pd.read_csv('raw-data/Egg_Production.csv')
poultry_df = pd.read_excel('raw-data/poultry_farm_dataset.xlsx')

egg_df
poultry_df

,Egg_Production,Mortality_Rate,Feed_Consumption,Weight_Growth,Revenue,Expenses,Temperature,Humidity,Disease_Incidence,Vaccination_Cost,Labor_Hours,Inventory_Feed,Inventory_Medication,Weather_Condition
0,917,0.012,5595,1.37,1108,986,28,0.54,Digestive,268,52,964,48,Sunny
1,1175,0.023,4676,1.14,1286,637,28,0.67,NaN,109,40,1093,54,Cloudy
2,904,0.026,4888,1.40,1701,801,21,0.70,Digestive,159,55,1130,51,Sunny
3,893,0.009,4899,1.52,1868,570,24,0.60,Respiratory,117,56,1011,54,Sunny
4,828,0.016,4553,1.78,1037,751,25,0.42,Respiratory,253,47,859,69,Sunny
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21846,869,0.011,4387,1.49,1864,644,29,0.52,Digestive,184,58,940,52,Cloudy
21847,1021,0.014,5344,1.61,1261,748,28,0.46,NaN,215,51,1046,46,Cloudy
21848,811,0.030,5833,1.26,1285,974,22,0.54,Respiratory,230,44,966,61,Clear
21849,980,0.009,4326,1.72,1735,632,20,0.67,Other,298,40,1103,55,Clear
